# Explode Glacier Polygons

This notebook explodes (merges) all glacier polygons that are touching to turn them into one polygon. Glaciers that are touching are part of a glacier catchment and I am interested in catchements as well as individual glaciers. Not all regions contain ice caps. The regions that have ice caps and that are processed in this notebook are the following:

* Region 3 - Arctic Canada, North
* Region 4 - Arctic Canada, South
* Region 5 - Greenland
* Region 6 - Iceland
* Region 7 - Svalbard and Jan Mayen
* Region 8 - Scandinavia
* Region 9 - Russian Arctic
* Region 10 - Asia, North
* Region 17 - Southern Andes

The ice cap and icefield outlines were created by looking at the GLIMS Viewer map (http://www.glims.org/maps/glims) and manaully selecting lat/lon bounding box coordinates around the ice caps.

In [1]:
import os
import os.path as op
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

# set working dir
HOME = op.join(op.expanduser("~"))
os.chdir(os.path.join(HOME, "git/wgms-glacier-project"))

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import scripts.wgms_scripts as ws

In [2]:
# Explode the regions that need to be exploded
regions = [3, 4, 5, 6, 7, 8, 9, 10, 17]
for i in regions:
    ws.explode_glaciers(i)

3
4
5
6
7
8
9
10
17
